In [9]:
!unzip archive.zip

Archive:  archive.zip
  inflating: 1000Folk_Story_around_the_Globe.csv  


In [2]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.8/431.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently 

In [14]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
OPENAI_KEY = userdata.get('OPENAI_KEY')

from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

In [4]:
import numpy as np
import pandas as pd
from openai import OpenAI

def getEmbedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = text, model=model).data[0].embedding

In [32]:
import pandas as pd

df = pd.read_csv('1000Folk_Story_around_the_Globe.csv')

In [13]:
max_length = 0
total = 0
for index, row in df.iterrows():
    text = row['full_text']
    if isinstance(text, str):  # Check if the value is a string
        max_length = max(max_length, len(text))
        total += len(text)

print(f"The length of the longest text in the 'full_text' column is: {max_length}")
print(total)

The length of the longest text in the 'full_text' column is: 75277
12985799


In [26]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

totalTokens = 0
count = 0
for index, row in df.iterrows():
    text = row['full_text']
    tokens = num_tokens_from_string(text, "cl100k_base")
    totalTokens += tokens

    if tokens > 8192:
        count += 1
        print(num_tokens_from_string(text, "cl100k_base"))
print(count)
print(totalTokens)

9070
10654
8594
12353
9643
11962
9621
8236
17515
8600
9787
8349
10099
9233
10931
9449
8414
8399
13973
11325
12443
10454
10587
13634
10669
25
3069578


In [34]:
df["tokenCount"] = df['full_text'].apply(lambda x: num_tokens_from_string(x, "cl100k_base"))

In [37]:
df = df[df["tokenCount"] < 8192]
df

,genre,source,region,title,full_text,tokenCount
0,Folktales,European folktales,British isles,Geraint and Enid,\nQueen Guinevere lay idly in bed dreaming bea...,6437
1,Folktales,European folktales,British isles,Lancelot and Elaine,\nHer name was Elaine. But she was so fair tha...,3363
2,Folktales,European folktales,British isles,Pelleas and Ettarde,\nFar away in a dreary land there lived a lad ...,3453
3,Folktales,European folktales,British isles,Gareth and Lynette,\nGareth was a little prince. His home was an ...,5846
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"\n\n ‘My strength is as the strength of ten, ...",3981
...,...,...,...,...,...,...
1301,Fairy tales,Andersen fairy tales,Denmark,Everything in its right place,"\nMore than a hundred years ago, behind the wo...",3859
1302,Fairy tales,Andersen fairy tales,Denmark,The real princess,\nThere was once a prince who wanted to marry ...,458
1303,Fairy tales,Andersen fairy tales,Denmark,The emperor's new clothes,\nMany years ago there was an emperor who was ...,2046
1304,Fairy tales,Andersen fairy tales,Denmark,Great Claus and little Claus,\nIn a village there once lived two men of the...,5287


In [38]:
df['embedding'] = df['full_text'].apply(lambda x: getEmbedding(x, model='text-embedding-3-small'))

<ipython-input-38-82e4731424da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'] = df['full_text'].apply(lambda x: getEmbedding(x, model='text-embedding-3-small'))


In [70]:
df.to_csv('FolkStoryEmbeddings.csv', index=False)

In [71]:
df = pd.read_csv('FolkStoryEmbeddings.csv')
df

,genre,source,region,title,full_text,tokenCount,embedding
0,Folktales,European folktales,British isles,Geraint and Enid,\nQueen Guinevere lay idly in bed dreaming bea...,6437,"[0.03761262819170952, 0.02196752093732357, -0...."
1,Folktales,European folktales,British isles,Lancelot and Elaine,\nHer name was Elaine. But she was so fair tha...,3363,"[0.030186014249920845, -0.010945448651909828, ..."
2,Folktales,European folktales,British isles,Pelleas and Ettarde,\nFar away in a dreary land there lived a lad ...,3453,"[0.011457417160272598, -0.006172125227749348, ..."
3,Folktales,European folktales,British isles,Gareth and Lynette,\nGareth was a little prince. His home was an ...,5846,"[0.005707132164388895, 0.026478230953216553, -..."
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"\n\n ‘My strength is as the strength of ten, ...",3981,"[-0.01282869465649128, 0.03587726503610611, -0..."
...,...,...,...,...,...,...,...
1276,Fairy tales,Andersen fairy tales,Denmark,Everything in its right place,"\nMore than a hundred years ago, behind the wo...",3859,"[0.027263831347227097, 0.019532956182956696, -..."
1277,Fairy tales,Andersen fairy tales,Denmark,The real princess,\nThere was once a prince who wanted to marry ...,458,"[0.006453713867813349, 0.014469905756413937, -..."
1278,Fairy tales,Andersen fairy tales,Denmark,The emperor's new clothes,\nMany years ago there was an emperor who was ...,2046,"[0.04256543889641762, 0.02121018059551716, -0...."
1279,Fairy tales,Andersen fairy tales,Denmark,Great Claus and little Claus,\nIn a village there once lived two men of the...,5287,"[0.03399132564663887, 0.014238215982913971, -0..."


In [76]:
import ast

def convert_to_list_of_floats(embedding_str):
    try:
        embedding_list = ast.literal_eval(embedding_str)
        return [float(x) for x in embedding_list]
    except (ValueError, SyntaxError):
        print(f"Error converting embedding: {embedding_str}")
        return []

df['embedding'] = df['embedding'].apply(convert_to_list_of_floats)

In [77]:
df.dtypes

,0
genre,object
source,object
region,object
title,object
full_text,object
tokenCount,int64
embedding,object


In [73]:
# prompt: remove unreadable chars and parse stuff in full_text col

import pandas as pd
import re

def clean_text(text):
    if isinstance(text, str):
        # Remove unreadable characters
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        # Additional cleaning if needed (e.g. remove extra whitespace)
        text = ' '.join(text.split())
        return text
    return text  # Return original value if not a string


df['full_text'] = df['full_text'].apply(clean_text)
df

,genre,source,region,title,full_text,tokenCount,embedding
0,Folktales,European folktales,British isles,Geraint and Enid,Queen Guinevere lay idly in bed dreaming beaut...,6437,"[0.03761262819170952, 0.02196752093732357, -0...."
1,Folktales,European folktales,British isles,Lancelot and Elaine,Her name was Elaine. But she was so fair that ...,3363,"[0.030186014249920845, -0.010945448651909828, ..."
2,Folktales,European folktales,British isles,Pelleas and Ettarde,Far away in a dreary land there lived a lad ca...,3453,"[0.011457417160272598, -0.006172125227749348, ..."
3,Folktales,European folktales,British isles,Gareth and Lynette,Gareth was a little prince. His home was an ol...,5846,"[0.005707132164388895, 0.026478230953216553, -..."
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"My strength is as the strength of ten, Because...",3981,"[-0.01282869465649128, 0.03587726503610611, -0..."
...,...,...,...,...,...,...,...
1276,Fairy tales,Andersen fairy tales,Denmark,Everything in its right place,"More than a hundred years ago, behind the wood...",3859,"[0.027263831347227097, 0.019532956182956696, -..."
1277,Fairy tales,Andersen fairy tales,Denmark,The real princess,There was once a prince who wanted to marry a ...,458,"[0.006453713867813349, 0.014469905756413937, -..."
1278,Fairy tales,Andersen fairy tales,Denmark,The emperor's new clothes,Many years ago there was an emperor who was so...,2046,"[0.04256543889641762, 0.02121018059551716, -0...."
1279,Fairy tales,Andersen fairy tales,Denmark,Great Claus and little Claus,In a village there once lived two men of the s...,5287,"[0.03399132564663887, 0.014238215982913971, -0..."


In [74]:
import re

def clean_text(text):
    if isinstance(text, str):
        # Remove backslashes
        text = text.replace("\"", "'")
        # Remove unreadable characters
        text = re.sub(r'[^\x00-\x7F]+', '', text)
        # Additional cleaning if needed (e.g. remove extra whitespace)
        text = ' '.join(text.split())
        return text
    return text  # Return original value if not a string


df['full_text'] = df['full_text'].apply(clean_text)
df

,genre,source,region,title,full_text,tokenCount,embedding
0,Folktales,European folktales,British isles,Geraint and Enid,Queen Guinevere lay idly in bed dreaming beaut...,6437,"[0.03761262819170952, 0.02196752093732357, -0...."
1,Folktales,European folktales,British isles,Lancelot and Elaine,Her name was Elaine. But she was so fair that ...,3363,"[0.030186014249920845, -0.010945448651909828, ..."
2,Folktales,European folktales,British isles,Pelleas and Ettarde,Far away in a dreary land there lived a lad ca...,3453,"[0.011457417160272598, -0.006172125227749348, ..."
3,Folktales,European folktales,British isles,Gareth and Lynette,Gareth was a little prince. His home was an ol...,5846,"[0.005707132164388895, 0.026478230953216553, -..."
4,Folktales,European folktales,British isles,Sir Galahad and the Sacred Cup,"My strength is as the strength of ten, Because...",3981,"[-0.01282869465649128, 0.03587726503610611, -0..."
...,...,...,...,...,...,...,...
1276,Fairy tales,Andersen fairy tales,Denmark,Everything in its right place,"More than a hundred years ago, behind the wood...",3859,"[0.027263831347227097, 0.019532956182956696, -..."
1277,Fairy tales,Andersen fairy tales,Denmark,The real princess,There was once a prince who wanted to marry a ...,458,"[0.006453713867813349, 0.014469905756413937, -..."
1278,Fairy tales,Andersen fairy tales,Denmark,The emperor's new clothes,Many years ago there was an emperor who was so...,2046,"[0.04256543889641762, 0.02121018059551716, -0...."
1279,Fairy tales,Andersen fairy tales,Denmark,Great Claus and little Claus,In a village there once lived two men of the s...,5287,"[0.03399132564663887, 0.014238215982913971, -0..."


In [78]:
df.to_json('/content/folkStoryEmbeddings.json', orient='records')